# 시계열 분류 모델링

이 노트북은 시계열 데이터에 대한 분류(Classification) 모델을 구현합니다.
시계열 데이터를 일정 길이의 윈도우로 나누고, 각 윈도우에서 추출한 특성(통계적, 주파수, 시차 등)을 바탕으로 해당 윈도우 또는 다음 시점의 상태(범주)를 예측하는 것을 목표로 합니다.
예를 들어, 센서 데이터 패턴을 기반으로 정상/비정상 상태를 분류하거나, 특정 시계열 패턴 발생 여부를 예측하는 문제에 적용할 수 있습니다.

## 사용 가이드
1. **데이터 설정**: `DATA_PATH`, `DATE_COL`, `TARGET_COL` 변수를 실제 데이터(타겟은 범주형 변수)에 맞게 수정합니다.
2. **특성 엔지니어링 설정**: 분류에 사용할 특성을 결정하기 위해 시계열 윈도우 크기(`WINDOW_SIZE`), 사용할 특성 종류(`USE_XXX_FEATURES`), 시차 값(`LAG_VALUES`) 등을 조정합니다.
3. **모델 선택**: 사용할 분류 모델(`MODELS`: RF, XGB, SVM)을 선택합니다.
4. **실행**: 노트북을 순차적으로 실행하여 시계열 분류 모델 결과를 확인합니다.

## 분석 흐름
1. **라이브러리 임포트 및 설정**: 필요한 라이브러리를 불러오고 기본 설정을 합니다.
2. **데이터 로드 및 탐색**: 데이터를 로드하고 기본적인 구조와 정보를 확인합니다.
3. **타겟 변수 탐색**: 분류할 대상(타겟 변수)의 분포와 시간적 변화를 탐색하고, 필요시 레이블 인코딩을 수행합니다.
4. **특성 변수 탐색**: 입력으로 사용할 초기 특성 변수들의 분포와 상관관계를 탐색합니다.
5. **특성 엔지니어링**: 시계열 데이터를 윈도우 기반으로 변환하고, 각 윈도우에서 통계적 특성, 주파수 특성, 시차 특성 등을 추출하여 모델 입력용 특성 집합을 생성합니다.
6. **데이터 분할 및 전처리**: 생성된 특성 데이터를 훈련, 검증, 테스트 세트로 분할하고 스케일링을 적용합니다. 클래스 불균형 여부도 확인합니다.
7. **모델 훈련 및 평가**: 선택된 분류 모델들을 훈련시키고 검증 데이터로 성능을 평가합니다. (평가지표: 정확도, 정밀도, 재현율, F1 점수, 혼동 행렬)
8. **모델 비교 및 결과 해석**: 최적 모델을 선정하고 테스트 데이터로 최종 성능을 평가합니다. (분류 보고서, 혼동 행렬, ROC 곡선 등 활용). 필요시 특성 중요도를 확인합니다.
9. **결론 및 인사이트**: 분석 결과와 모델 성능을 요약하고 인사이트를 도출합니다.

## 1. 사용자 입력 파라미터 설정
분석 데이터, 특성 생성 방식, 사용할 모델 등을 설정합니다.
특히 시계열 분류에서는 어떤 정보를 특성으로 사용할지가 중요하므로, 특성 엔지니어링 관련 파라미터 설정이 중요합니다.

In [ ]:
# ===== 필수 수정 파라미터 =====
# 데이터 파일 경로 (CSV 형식 권장)
DATA_PATH = "../data/raw/your_data.csv"

# 날짜 열 이름 (데이터프레임 내 날짜/시간 정보가 있는 열)
DATE_COL = "date"

# 타겟 열 이름 (분류하려는 범주형 변수가 있는 열)
TARGET_COL = "target"

# ===== 선택적 수정 파라미터 =====
# 특성 엔지니어링 파라미터
WINDOW_SIZE = 24                       # 시계열 윈도우 크기
USE_STATISTICAL_FEATURES = True        # 통계적 특성 사용 여부
USE_FREQUENCY_FEATURES = True          # 주파수 도메인 특성 사용 여부
USE_LAG_FEATURES = True                # 지연(lag) 특성 사용 여부
LAG_VALUES = [1, 7, 14]                # 사용할 지연 값들

# 모델 파라미터
MODELS = ["RF", "XGB", "SVM"]         # 사용할 모델들 (RF: RandomForest, XGB: XGBoost, SVM: Support Vector Machine)
TRAIN_SIZE = 0.7                       # 훈련 데이터 비율
VALIDATION_SIZE = 0.15                 # 검증 데이터 비율 (나머지는 테스트 데이터)
RANDOM_STATE = 42                      # 랜덤 시드

# 출력 경로
OUTPUT_DIR = '../plots'          # 시각화 결과 저장 경로
DATA_OUTPUT_DIR = '../data/processed' # 전처리된 데이터 저장 경로

# 출력 디렉토리 생성
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DATA_OUTPUT_DIR, exist_ok=True)

## 2. 필요 라이브러리 임포트
데이터 처리, 시각화, 특성 추출(FFT 등), 머신러닝 분류 모델 및 평가에 필요한 라이브러리를 불러옵니다.

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.fft import fft

# 머신러닝 라이브러리
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, 
                           accuracy_score, precision_score, recall_score, f1_score,
                           roc_curve, auc, roc_auc_score)

# 분류 모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# 경고 무시
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

# 랜덤 시드 설정
np.random.seed(RANDOM_STATE)

## 3. 데이터 로드 및 탐색
지정된 경로에서 데이터를 로드하고, 날짜 컬럼을 인덱스로 설정하며 기본적인 정보를 확인합니다.

In [ ]:
# 데이터 로드
df = pd.read_csv(DATA_PATH)
print(f"데이터 로드 완료. 형태: {df.shape}")

# 날짜 열 처리
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df.set_index(DATE_COL, inplace=True)
print(f"'{DATE_COL}' 열을 날짜 형식으로 변환하고 인덱스로 설정했습니다.")

# 데이터 정보 출력
print("\n데이터 기본 정보:")
print(df.info())
print("\n기술 통계량:")
print(df.describe())

# 처음 몇 행 확인
print("\n처음 5개 행:")
print(df.head())

## 4. 타겟 변수 탐색
분류 문제의 대상이 되는 타겟 변수(`TARGET_COL`)의 특성을 파악합니다.
- **클래스 분포 확인**: 각 클래스(범주)에 속하는 데이터의 개수를 확인하여 분포를 파악하고 시각화합니다. 클래스 불균형이 심한지 확인합니다.
- **시간에 따른 분포**: 시간에 따라 타겟 클래스가 어떻게 변화하는지 시각화합니다.
- **레이블 인코딩**: 타겟 변수가 문자열 등 범주형일 경우, 머신러닝 모델이 처리할 수 있도록 숫자 형태로 변환(인코딩)합니다.

In [ ]:
# 타겟 변수 확인
target_values = df[TARGET_COL].value_counts()
print(f"\n타겟 변수({TARGET_COL}) 분포:")
print(target_values)

# 타겟 변수 분포 시각화
plt.figure(figsize=(10, 6))
target_values.plot(kind='bar')
plt.title(f'타겟 변수({TARGET_COL}) 분포')
plt.xlabel('클래스')
plt.ylabel('빈도')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/class_타겟변수분포.png')
plt.show()

# 타겟 변수의 시계열 시각화
plt.figure(figsize=(14, 7))
for class_value in df[TARGET_COL].unique():
    class_data = df[df[TARGET_COL] == class_value]
    plt.scatter(class_data.index, [class_value] * len(class_data), 
                alpha=0.6, label=f'클래스 {class_value}')
plt.title(f'시간에 따른 {TARGET_COL} 클래스 분포')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/class_시간에따른분포.png')
plt.show()

# 타겟 변수가 범주형이 아닌 경우, 레이블 인코딩
if df[TARGET_COL].dtype not in ['int64', 'int32', 'uint8', 'bool']:
    print(f"\n타겟 변수({TARGET_COL})가 범주형이 아닌 것으로 판단됩니다. 레이블 인코딩을 수행합니다.")
    label_encoder = LabelEncoder()
    df[TARGET_COL] = label_encoder.fit_transform(df[TARGET_COL])
    print(f"레이블 인코딩 결과: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

## 5. 특성 변수 탐색
모델 입력으로 사용될 초기 특성 변수들(타겟 변수 제외)의 분포와 상호 관계를 탐색합니다.
- **특성 분포 시각화**: 각 특성 값의 분포를 히스토그램으로 확인합니다.
- **상관관계 분석**: 특성 변수들 간의 선형 상관관계를 히트맵으로 시각화하여 다중공선성 등을 검토합니다.
- **시계열 특성 시각화**: 주요 특성 변수들이 시간에 따라 어떻게 변화하는지 시각화합니다.

In [ ]:
# 특성 선택 (타겟 변수 제외한 모든 열)
features = [col for col in df.columns if col != TARGET_COL]
print(f"\n선택된 특성({len(features)}개): {features}")

# 기본 특성 탐색
if features:
    # 히스토그램으로 특성 분포 시각화
    feature_data = df[features]
    feature_data.hist(bins=20, figsize=(14, 10))
    plt.suptitle('특성 분포 히스토그램')
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.savefig(f'{OUTPUT_DIR}/class_특성분포.png')
    plt.show()
    
    # 상관관계 분석
    plt.figure(figsize=(12, 10))
    correlation = feature_data.corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))
    sns.heatmap(correlation, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                square=True, linewidths=.5)
    plt.title('특성 간 상관관계')
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/class_특성상관관계.png')
    plt.show()
    
    # 시계열 특성 시각화 (처음 3개 특성만)
    plt.figure(figsize=(14, 10))
    for i, feature in enumerate(features[:min(3, len(features))]):
        plt.subplot(min(3, len(features)), 1, i+1)
        plt.plot(df.index, df[feature])
        plt.title(f'시계열 특성: {feature}')
        plt.xlabel('날짜')
        plt.ylabel(feature)
        plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/class_시계열특성.png')
    plt.show()

## 6. 특성 엔지니어링
시계열 데이터를 고정 길이의 윈도우로 분할하고, 각 윈도우 내에서 다양한 특성을 추출하여 분류 모델의 입력 데이터를 생성합니다.
이 과정은 원본 시계열의 시간적 패턴 정보를 요약된 특성 벡터로 변환하는 핵심 단계입니다.
- **통계적 특성 추출 (`extract_statistical_features`)**: 윈도우 내 데이터의 평균, 표준편차, 최소/최대값 등 통계량을 계산합니다.
- **주파수 특성 추출 (`extract_frequency_features`)**: 윈도우 내 데이터에 FFT(고속 푸리에 변환)를 적용하여 주파수 도메인의 특성(평균, 표준편차, 최대값 등)을 추출합니다. 주기적 패턴 분석에 유용할 수 있습니다.
- **지연 특성 추출 (`extract_lag_features`)**: 윈도우의 마지막 시점을 기준으로 과거 시점의 값(lag)을 특성으로 사용합니다.
- **특성 집합 생성 (`create_feature_set`)**: 위 함수들을 이용하여 각 윈도우별로 모든 특성을 계산하고, 해당 윈도우 다음 시점의 타겟 값과 연결하여 최종 특성 데이터셋을 만듭니다.

In [ ]:
def extract_statistical_features(series):
    """시계열 데이터에서 통계적 특성을 추출합니다."""
    stats_features = {}
    
    # 기본 통계량
    stats_features['mean'] = series.mean()
    stats_features['std'] = series.std()
    stats_features['min'] = series.min()
    stats_features['max'] = series.max()
    stats_features['median'] = series.median()
    
    # 백분위수
    stats_features['q25'] = series.quantile(0.25)
    stats_features['q75'] = series.quantile(0.75)
    stats_features['iqr'] = stats_features['q75'] - stats_features['q25']
    
    # 왜도와 첨도
    stats_features['skew'] = series.skew()
    stats_features['kurtosis'] = series.kurtosis()
    
    return stats_features

def extract_frequency_features(series):
    """시계열 데이터에서 주파수 도메인 특성을 추출합니다."""
    freq_features = {}
    
    # FFT 변환
    fft_values = fft(series.values)
    fft_abs = np.abs(fft_values)
    
    # 주요 주파수 특성
    freq_features['fft_mean'] = np.mean(fft_abs)
    freq_features['fft_std'] = np.std(fft_abs)
    freq_features['fft_max'] = np.max(fft_abs)
    
    # 주파수 스펙트럼의 상위 5개 값
    top_indices = np.argsort(fft_abs)[-5:]
    for i, idx in enumerate(top_indices):
        freq_features[f'fft_top{i+1}'] = fft_abs[idx]
    
    return freq_features

def extract_lag_features(series, lags):
    """시계열 데이터에서 지연(lag) 특성을 추출합니다."""
    lag_features = {}
    
    for lag in lags:
        lag_features[f'lag_{lag}'] = series.shift(lag).iloc[-1]
    
    return lag_features

# 윈도우 기반 특성 추출
def create_feature_set(df, features, window_size, target_col, 
                       use_statistical=True, use_frequency=True, use_lag=True, lag_values=None):
    """윈도우 기반의 특성 집합을 생성합니다."""
    feature_sets = []
    
    # 날짜 범위의 끝에서부터 특성 추출 시작
    for end_idx in range(window_size, len(df) + 1):
        window_data = df.iloc[end_idx - window_size:end_idx]
        features_dict = {}
        
        # 각 특성별로 윈도우 특성 추출
        for feature in features:
            series = window_data[feature]
            feature_prefix = f"{feature}_"
            
            # 통계적 특성
            if use_statistical:
                stat_features = extract_statistical_features(series)
                for key, value in stat_features.items():
                    features_dict[feature_prefix + key] = value
            
            # 주파수 특성
            if use_frequency:
                freq_features = extract_frequency_features(series)
                for key, value in freq_features.items():
                    features_dict[feature_prefix + key] = value
            
            # 지연 특성
            if use_lag and lag_values:
                lag_features = extract_lag_features(df[feature].iloc[:end_idx], lag_values)
                for key, value in lag_features.items():
                    features_dict[feature_prefix + key] = value
        
        # 타겟 값
        if end_idx < len(df):
            features_dict['target'] = df[target_col].iloc[end_idx]
        
        feature_sets.append(features_dict)
    
    # 마지막 윈도우는 예측용이므로 타겟이 없음
    feature_sets = feature_sets[:-1]
    
    return pd.DataFrame(feature_sets)

## 7. 특성 집합 생성
정의된 함수를 사용하여 실제 특성 엔지니어링을 수행하고, 생성된 특성 데이터셋을 확인합니다.
특성 생성 과정에서 발생할 수 있는 결측치는 평균값으로 대체합니다.

In [ ]:
print("윈도우 기반 특성 집합을 생성합니다...")
features_df = create_feature_set(
    df, features, WINDOW_SIZE, TARGET_COL,
    use_statistical=USE_STATISTICAL_FEATURES,
    use_frequency=USE_FREQUENCY_FEATURES,
    use_lag=USE_LAG_FEATURES,
    lag_values=LAG_VALUES
)

print(f"생성된 특성 집합 형태: {features_df.shape}")
print("\n생성된 특성 목록 (처음 10개):")
feature_cols = [col for col in features_df.columns if col != 'target']
for col in feature_cols[:10]:
    print(f"- {col}")
if len(feature_cols) > 10:
    print(f"... 외 {len(feature_cols) - 10}개")

# 데이터 확인
print("\n특성 집합 처음 5개 행:")
print(features_df.head())

# 결측치 처리
missing_values = features_df.isnull().sum()
if missing_values.sum() > 0:
    print("\n결측치가 있는 특성 수:", (missing_values > 0).sum())
    
    # 결측치 처리 (평균으로 대체)
    features_df = features_df.fillna(features_df.mean())
    print("결측치를 평균값으로 대체했습니다.")

## 8. 데이터 분할 및 전처리
생성된 특성 데이터셋을 모델 학습과 평가를 위해 훈련, 검증, 테스트 세트로 분할합니다.
시계열 데이터의 특성을 고려하여 시간 순서를 유지하며 분할하는 대신, 여기서는 일반적인 분류 문제처럼 랜덤 분할(`train_test_split`)을 사용합니다. (필요시 시간 순서 분할 방식으로 변경 가능)
특성 스케일링(`StandardScaler`)을 적용하여 모델 학습 성능을 높입니다.
훈련 데이터의 클래스 분포를 확인하여 불균형 문제를 인지합니다.

In [ ]:
# 특성과 타겟 분리
X = features_df.drop('target', axis=1)
y = features_df['target']

# 데이터 분할 (훈련, 검증, 테스트)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=(1 - TRAIN_SIZE - VALIDATION_SIZE), random_state=RANDOM_STATE
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=VALIDATION_SIZE/(TRAIN_SIZE + VALIDATION_SIZE), 
    random_state=RANDOM_STATE
)

print(f"데이터 분할 완료:")
print(f"- 훈련 데이터: {X_train.shape}")
print(f"- 검증 데이터: {X_val.shape}")
print(f"- 테스트 데이터: {X_test.shape}")

# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n특성 스케일링 완료")

# 레이블 분포 확인
train_class_counts = pd.Series(y_train).value_counts()
print("\n훈련 데이터 클래스 분포:")
print(train_class_counts)

# 불균형 확인
if len(train_class_counts) > 1:  # 클래스가 2개 이상인 경우
    max_class = train_class_counts.max()
    min_class = train_class_counts.min()
    imbalance_ratio = max_class / min_class
    
    print(f"\n클래스 불균형 비율: {imbalance_ratio:.2f}")
    if imbalance_ratio > 3:
        print("경고: 클래스 불균형이 크게 발생했습니다. 균형 조정 기법 사용을 고려하세요.")

## 9. 모델 훈련 및 평가
선택된 분류 모델(Random Forest, XGBoost, SVM)들을 훈련 데이터로 학습시키고, 검증 데이터로 성능을 평가하여 최적 모델을 찾습니다.
- **모델 정의**: 각 분류 모델 객체를 생성합니다.
- **훈련 및 평가**: 각 모델을 훈련(`fit`)하고 검증 세트로 예측(`predict`)하여 성능 지표(정확도, 정밀도, 재현율, F1 점수)를 계산합니다. 혼동 행렬을 시각화하여 분류 성능을 상세히 분석합니다.
- **최적 모델 선정**: 검증 세트에서의 F1 점수를 기준으로 가장 성능이 좋은 모델을 선택합니다.
- **최종 평가**: 선택된 최적 모델을 사용하여 테스트 세트에 대한 최종 성능을 평가하고, 분류 보고서, 혼동 행렬, ROC 곡선(이진 분류 시) 등을 통해 결과를 제시합니다. Tree 기반 모델의 경우 특성 중요도도 확인합니다.

In [ ]:
# 모델 사전 정의
models = {}

if "RF" in MODELS:
    models["RandomForest"] = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        random_state=RANDOM_STATE
    )

if "XGB" in MODELS:
    models["XGBoost"] = XGBClassifier(
        n_estimators=100, 
        max_depth=5, 
        learning_rate=0.1,
        random_state=RANDOM_STATE
    )

if "SVM" in MODELS:
    models["SVM"] = SVC(
        kernel='rbf', 
        probability=True,
        random_state=RANDOM_STATE
    )

# 모델 훈련 및 평가
model_results = {}
for name, model in models.items():
    print(f"\n{name} 모델 훈련 중...")
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_val_pred = model.predict(X_val_scaled)
    
    # 성능 평가
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='weighted')
    recall = recall_score(y_val, y_val_pred, average='weighted')
    f1 = f1_score(y_val, y_val_pred, average='weighted')
    
    print(f"{name} 검증 성능:")
    print(f"- 정확도: {accuracy:.4f}")
    print(f"- 정밀도: {precision:.4f}")
    print(f"- 재현율: {recall:.4f}")
    print(f"- F1 점수: {f1:.4f}")
    
    # 혼동 행렬
    cm = confusion_matrix(y_val, y_val_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} 혼동 행렬')
    plt.xlabel('예측 클래스')
    plt.ylabel('실제 클래스')
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/class_{name}_혼동행렬.png')
    plt.show()
    
    # 결과 저장
    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# 최고 성능 모델 선택
if model_results:
    best_model_name = max(model_results, key=lambda x: model_results[x]['f1'])
    best_model = model_results[best_model_name]['model']
    
    print(f"\n최고 성능 모델: {best_model_name}")
    print(f"검증 F1 점수: {model_results[best_model_name]['f1']:.4f}")
    
    # 최고 모델로 테스트 데이터 평가
    y_test_pred = best_model.predict(X_test_scaled)
    
    # 테스트 성능 평가
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    print(f"\n{best_model_name} 테스트 성능:")
    print(f"- 정확도: {test_accuracy:.4f}")
    print(f"- 정밀도: {test_precision:.4f}")
    print(f"- 재현율: {test_recall:.4f}")
    print(f"- F1 점수: {test_f1:.4f}")
    
    # 분류 보고서
    print("\n분류 보고서:")
    print(classification_report(y_test, y_test_pred))
    
    # 최종 혼동 행렬
    cm_test = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{best_model_name} 테스트 혼동 행렬')
    plt.xlabel('예측 클래스')
    plt.ylabel('실제 클래스')
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/class_최종_혼동행렬.png')
    plt.show()
    
    # 이진 분류인 경우 ROC 곡선
    if len(np.unique(y)) == 2:
        y_test_prob = best_model.predict_proba(X_test_scaled)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_test_prob)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC 곡선 (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC 곡선')
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/class_ROC곡선.png')
        plt.show()
        
    # 특성 중요도 (RandomForest와 XGBoost의 경우)
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        plt.figure(figsize=(12, 8))
        sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
        plt.title(f'{best_model_name} 특성 중요도 (상위 20개)')
        plt.xlabel('중요도')
        plt.ylabel('특성')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_DIR}/class_특성중요도.png')
        plt.show()
        
        print("\n상위 10개 중요 특성:")
        print(feature_importance.head(10))

## 10. 결론 및 인사이트
전체 시계열 분류 모델링 과정과 최종 결과를 요약합니다.
데이터 정보, 사용된 특성 수, 각 모델의 검증 성능, 최적 모델의 최종 테스트 성능 등을 제시하고 분석에 대한 결론과 인사이트를 정리합니다.

In [ ]:
print("\n" + "="*50)
print("시계열 분류 모델링 요약")
print("="*50)

print(f"\n1. 데이터 정보:")
print(f"   - 데이터 기간: {df.index.min()} ~ {df.index.max()}")
print(f"   - 데이터 포인트 수: {len(df)}")
print(f"   - 특성 수: {len(features)}")
print(f"   - 추출된 특성 수: {X.shape[1]}")

print(f"\n2. 모델 성능:")
for model_name, results in model_results.items():
    print(f"   - {model_name}: 검증 F1 = {results['f1']:.4f}, 정확도 = {results['accuracy']:.4f}")

print(f"\n3. 최고 성능 모델: {best_model_name}")
print(f"   - 테스트 F1: {test_f1:.4f}")
print(f"   - 테스트 정확도: {test_accuracy:.4f}")

print("\n시계열 분류 모델링이 완료되었습니다!")